<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Constanza Peña

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno: Camila J. Pulgar Fernández



### **Link de repositorio de GitHub:** `https://github.com/cpulgarf/Laboratorios/`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Exploración del dataset y los resultados a través de visualizaciones interactivas usando `plotly`.
- Introducción a procesamiento del lenguaje natural.
- Clasificación en `scikit-learn`
- Uso de pipelines.

## Reglas:

- Fecha de entrega: 26/11/2021
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Aplicar las ventajas que nos ofrece crear un pipeline.
- Obtener caracteristicas desde texto.
- Visualizar el funcionamiento de clasificadores.
- Realizar una GridSearch sobre un conjunto de clasificadores.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [2]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')

     |████████████████████████████████| 25.3 MB 1.9 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
     |████████████████████████████████| 86 kB 2.9 MB/s 
     |████████████████████████████████| 1.1 MB 27.5 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82709 sha256=129b8902aaea26849c1fa18dbef13fdeff0423f5f2e7d9af09c08f6e80d68dd1
  Stored in directory: /root/.cache/pip/wheels/84/1b/c6/aaf68a748122632967cef4dffef68224eb16798b6793257d82
  Created wheel for pynndescent: filename=pynndescent-0.5.5-py3-none-any.whl size=52603 sha256=240da0f01a04fa3f58bfdcb7a58f0af9f49ca02b1cf529904a63ba460259d584
  Stored in directory: /root/.cache/pip/wheels/af/e9/33/04db1436df0757c42fda8ea6796d7a8586e23c85fac355f476
Successfully built umap-learn pynndescent
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwo

True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [75]:
df_comics = pd.read_csv('df_comics.csv', index_col=[0])
df_comics_no_label = pd.read_csv('comics_no_label.csv', index_col=[0])

In [81]:
def exploratory_data_analysis(dataframe):
     print('1.- El DataFrame tiene ' + str(dataframe.shape[0]) + ' filas y ' + str(dataframe.shape[1])+ ' columnas')
     print('\n')
     print('2.- El DataFrame esta compuesto por las siguientes columnas:' + str(list(dataframe.columns)) )
     print('\n')
     print('3.- Ejemplos de filas del DataFrame:'  + '\n ' + 'Primeras 5 filas')
     print('\n')
     display(dataframe.head(5))
     print('\n')
     print('Últimas 5 filas')
     print('\n')
     display(dataframe.tail(5))
     print('\n')
     print('Muestreo aleatorio de 5 filas:')
     display(dataframe.sample(5))
     print('4.- Descripción numérica del Dataframe:')
     display(dataframe.describe())
     print('5.- Cantidad de valores nulos por columna:')
     display(dataframe.isna().sum())
     print('\n')
     print('6.- Cantidad de valores únicos por columna:')
     print('\n')
     display(dataframe.nunique())

In [82]:
# queda a labor de su equipo hacer el análisis exploratorio
exploratory_data_analysis(df_comics )

1.- El DataFrame tiene 1367 filas y 81 columnas


2.- El DataFrame esta compuesto por las siguientes columnas:['name', 'real_name', 'full_name', 'overall_score', 'history_text', 'powers_text', 'intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score', 'superpowers', 'alter_egos', 'aliases', 'place_of_birth', 'first_appearance', 'creator', 'alignment', 'occupation', 'base', 'teams', 'relatives', 'gender', 'type_race', 'height', 'weight', 'eye_color', 'hair_color', 'skin_color', 'img', 'has_electrokinesis', 'has_energy_constructs', 'has_mind_control_resistance', 'has_matter_manipulation', 'has_telepathy_resistance', 'has_mind_control', 'has_enhanced_hearing', 'has_dimensional_travel', 'has_element_control', 'has_size_changing', 'has_fire_resistance', 'has_fire_control', 'has_dexterity', 'has_reality_warping', 'has_illusions', 'has_energy_beams', 'has_peak_human_condition', 'has_shapeshifting', 'has_heat_resistance', 'has_jump', 'has_self-sus

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,alter_egos,aliases,place_of_birth,first_appearance,creator,alignment,occupation,base,teams,relatives,gender,type_race,height,weight,eye_color,hair_color,skin_color,img,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,has_matter_manipulation,has_telepathy_resistance,has_mind_control,has_enhanced_hearing,has_dimensional_travel,has_element_control,...,has_fire_resistance,has_fire_control,has_dexterity,has_reality_warping,has_illusions,has_energy_beams,has_peak_human_condition,has_shapeshifting,has_heat_resistance,has_jump,has_self-sustenance,has_energy_absorption,has_cold_resistance,has_magic,has_telekinesis,has_toxin_and_disease_resistance,has_telepathy,has_regeneration,has_immortality,has_teleportation,has_force_fields,has_energy_manipulation,has_endurance,has_longevity,has_weapon-based_powers,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,60,40,70,"['Super Speed', 'Super Strength']",[],[''],NaN,NaN,Marvel Comics,Good,NaN,NaN,"['Annihilators', 'Asgardians', 'Avengers', 'Ne...",NaN,Male,Human,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/10038.jpg?v=156096...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,100,80,"['Accelerated Healing', 'Agility', 'Berserk Mo...",[],['Rick Jones'],"Scarsdale, Arizona","Hulk Vol 2 #2 (April, 2008) (as A-Bomb)",Marvel Comics,Good,"Musician, adventurer, author; formerly talk sh...",NaN,"['Teen Brigade', 'Ultimate Fantastic Four', 'U...",Marlo Chandler-Jones (wife); Polly (aunt); Mrs...,Male,Human,6'8 • 203 cm,980 lb • 441 kg,Yellow,No Hair,NaN,/pictures2/portraits/10/050/10060.jpg?v=158233...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,100,55,"['Energy Absorption', 'Energy Armor', 'Energy ...",[],[''],Stoneworld,Green Lantern Vol 3 #21,DC Comics,Good,NaN,NaN,"['Blue Lantern Corps', 'Green Lantern Corps', ...",NaN,Male,Human,-,-,NaN,NaN,NaN,/pictures2/portraits/10/050/1410.jpg?v=1581168103,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,40,30,50,"['Weapon-based Powers', 'Weapons Master']",[],[''],Gotham City,NaN,DC Comics,Good,NaN,NaN,[],NaN,Male,Human,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/11650.jpg?v=156173...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,55,55,85,"['Accelerated Healing', 'Agility', 'Astral Pro...",[],[''],NaN,NaN,George Lucas,Good,NaN,NaN,['Jedi Order'],NaN,Female,Twi'lek,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/10891.jpg?v=156181...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,



Últimas 5 filas




,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,alter_egos,aliases,place_of_birth,first_appearance,creator,alignment,occupation,base,teams,relatives,gender,type_race,height,weight,eye_color,hair_color,skin_color,img,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,has_matter_manipulation,has_telepathy_resistance,has_mind_control,has_enhanced_hearing,has_dimensional_travel,has_element_control,...,has_fire_resistance,has_fire_control,has_dexterity,has_reality_warping,has_illusions,has_energy_beams,has_peak_human_condition,has_shapeshifting,has_heat_resistance,has_jump,has_self-sustenance,has_energy_absorption,has_cold_resistance,has_magic,has_telekinesis,has_toxin_and_disease_resistance,has_telepathy,has_regeneration,has_immortality,has_teleportation,has_force_fields,has_energy_manipulation,has_endurance,has_longevity,has_weapon-based_powers,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
1445,Zatanna,Zatanna Zatara,Zatanna Zatara,10,Zatanna is the daughter of adventurer John Zat...,Zatanna is genetically talented with her magi...,90,10,25,30,100,55,"['Cryokinesis', 'Fire Control', 'Magic', 'Prob...","['Chaos Zatanna', 'Doctor Fate', 'Zatanna (Ful...",[''],NaN,Hawkman #4,DC Comics,Good,NaN,NaN,"['Seven Soldiers of Victory', 'Sentinels of Ma...","Giovanni ""John"" Zatara (father, deceased), Sin...",Female,Human,5'7 • 170 cm,127 lb • 57 kg,Blue,Black,NaN,/pictures2/portraits/10/050/809.jpg?v=1578913192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1446,Zero,DWN-∞: Zero,DWN-∞: Zero,18,Zero was created by the late Dr. Albert Wily ...,NaN,80,100,100,100,100,80,"['Accelerated Healing', 'Acrobatics', 'Agility...",[],[''],NaN,Mega Man X (1993),Capcom,Good,NaN,NaN,[],NaN,Male,Robot,5'6 • 168 cm,145 lb • 65 kg,Blue,Blond,Red,/pictures2/portraits/10/050/12699.jpg?v=158402...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1447,Zoom (New 52),Hunter Zolomon,NaN,20,"Hunter Zolomon is better known as Zoom, a spee...",After tricking Barry Allen and Wally West into...,95,50,100,75,100,80,"['Accelerated Healing', 'Agility', 'Durability...","['Black Flash (CW)', 'Zoom', 'Zoom (CW)']",['Judge · Reverse-Flash · The Flash'],NaN,Flash Secret Files and Origins #3,DC Comics,Bad,"Criminal · former F.B.I. Profiler, Politician","Keystone City, Kansas",['Flash Family'],Ashley Zolomon (ex-wife),Male,NaN,6'1 • 185 cm,181 lb • 81 kg,Red,Brown,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1448,Zoom,Hunter Zolomon,Hunter Zolomon,9,Hunter Zolomon had a troubled relationship wi...,"Zoom is able to alter time, to make himself ev...",75,10,100,30,100,30,"['Intangibility', 'Super Speed', 'Time Manipul...","['Black Flash (CW)', 'Zoom (CW)', 'Zoom (New 5...",[''],NaN,Flash Secret Files #3,DC Comics,Bad,NaN,"Keystone City, Kansas","['Rogues', 'The Society', 'Flash Family', 'Jus...",Ashley Zolomon (ex-wife),Male,NaN,6'1 • 185 cm,181 lb • 81 kg,Red,Brown,NaN,/pictures2/portraits/10/050/1384.jpg?v=1295759009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1449,Zzzax,NaN,NaN,5,Zzzax was the product of an accident that occu...,NaN,45,80,75



Muestreo aleatorio de 5 filas:


...

,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,has_matter_manipulation,has_telepathy_resistance,has_mind_control,has_enhanced_hearing,has_dimensional_travel,has_element_control,has_size_changing,has_fire_resistance,has_fire_control,has_dexterity,has_reality_warping,has_illusions,has_energy_beams,has_peak_human_condition,has_shapeshifting,has_heat_resistance,has_jump,has_self-sustenance,has_energy_absorption,has_cold_resistance,has_magic,has_telekinesis,has_toxin_and_disease_resistance,has_telepathy,has_regeneration,has_immortality,has_teleportation,has_force_fields,has_energy_manipulation,has_endurance,has_longevity,has_weapon-based_powers,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
count,1367.000000,1367.000000,1367.000000,1367.000000,1367.000000,1367.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000,1306.000000
mean,78.899049,39.374543,46.313094,56.079005,67.019020,68.076079,0.089587,0.078867,0.075804,0.050536,0.083461,0.084227,0.137060,0.084227,0.104900,0.091118,0.118683,0.094181,0.153905,0.049770,0.094181,0.096478,0.171516,0.117152,0.151608,0.154671,0.125574,0.145482,0.149311,0.107963,0.106432,0.154671,0.139357,0.150842,0.147779,0.139357,0.137825,0.143185,0.235069,0.231240,0.260337,0.209035,0.246554,0.254977,0.329250,0.355283,0.271822,0.312404,0.407351,0.439510,0.437979,0.395100,0.512251,0.543645,0.571975,0.529862
std,25.120035,34.579573,29.434246,31.843475,33.240412,29.662451,0.285698,0.269634,0.264786,0.219132,0.276684,0.277834,0.344042,0.277834,0.306542,0.287887,0.323539,0.292192,0.360996,0.217553,0.292192,0.295358,0.377104,0.321724,0.358778,0.361729,0.331496,0.352722,0.356531,0.310453,0.308508,0.361729,0.346451,0.358032,0.355017,0.346451,0.344849,0.350396,0.424204,0.421787,0.438987,0.406775,0.431170,0.436015,0.470121,0.478782,0.445069,0.463651,0.491529,0.496518,0.496328,0.489059,0.500041,0.498282,0.494982,0.499299
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,75.000000,10.000000,25.000000,30.000000,40.000000,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,85.000000,30.000000,40.000000,55.000000,75.000000,75.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

5.- Cantidad de valores nulos por columna:


name                    0
real_name             131
full_name             423
overall_score           0
history_text           82
                     ... 
has_super_speed        61
has_durability         61
has_stamina            61
has_agility            61
has_super_strength     61
Length: 81, dtype: int64



6.- Cantidad de valores únicos por columna:




name                  1367
real_name             1025
full_name              832
overall_score           93
history_text          1274
                      ... 
has_super_speed          2
has_durability           2
has_stamina              2
has_agility              2
has_super_strength       2
Length: 81, dtype: int64

Como se quiere analizar los textos, es necesario que se tengan datos sobre estos, por lo que se eliminarán aquellos registros en los que no hay información en "History Text"

In [86]:
df_comics = df_comics.dropna(subset=['history_text']) 
df_comics = df_comics.drop_duplicates()

In [6]:
df_comics.head(3)

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,alter_egos,aliases,place_of_birth,first_appearance,creator,alignment,occupation,base,teams,relatives,gender,type_race,height,weight,eye_color,hair_color,skin_color,img,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,has_matter_manipulation,has_telepathy_resistance,has_mind_control,has_enhanced_hearing,has_dimensional_travel,has_element_control,...,has_fire_resistance,has_fire_control,has_dexterity,has_reality_warping,has_illusions,has_energy_beams,has_peak_human_condition,has_shapeshifting,has_heat_resistance,has_jump,has_self-sustenance,has_energy_absorption,has_cold_resistance,has_magic,has_telekinesis,has_toxin_and_disease_resistance,has_telepathy,has_regeneration,has_immortality,has_teleportation,has_force_fields,has_energy_manipulation,has_endurance,has_longevity,has_weapon-based_powers,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,60,40,70,"['Super Speed', 'Super Strength']",[],[''],NaN,NaN,Marvel Comics,Good,NaN,NaN,"['Annihilators', 'Asgardians', 'Avengers', 'Ne...",NaN,Male,Human,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/10038.jpg?v=156096...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,100,100,80,"['Accelerated Healing', 'Agility', 'Berserk Mo...",[],['Rick Jones'],"Scarsdale, Arizona","Hulk Vol 2 #2 (April, 2008) (as A-Bomb)",Marvel Comics,Good,"Musician, adventurer, author; formerly talk sh...",NaN,"['Teen Brigade', 'Ultimate Fantastic Four', 'U...",Marlo Chandler-Jones (wife); Polly (aunt); Mrs...,Male,Human,6'8 • 203 cm,980 lb • 441 kg,Yellow,No Hair,NaN,/pictures2/portraits/10/050/10060.jpg?v=158233...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,45,100,55,"['Energy Absorption', 'Energy Armor', 'Energy ...",[],[''],Stoneworld,Green Lantern Vol 3 #21,DC Comics,Good,NaN,NaN,"['Blue Lantern Corps', 'Green Lantern Corps', ...",NaN,Male,Human,-,-,NaN,NaN,NaN,/pictures2/portraits/10/050/1410.jpg?v=1581168103,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `bag of words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `bag of words`?...

`bag of words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `bag of words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `bag of words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [ ]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [ ]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('spanish')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'beauti', 'citi']]

In [ ]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [ ]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,in,is,like,new,rock,roll,teacher,the,world,york
0,1,0,0,0,0,1,0,0,1,0,2,1,1,1,0,0
1,0,1,0,1,0,0,1,1,0,0,1,0,0,3,1,0
2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [ ]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor in,beauti,beauti citi,best,best actor,citi,good,...,teacher,teacher rock,the,the best,the rock,the teacher,the world,world,york,york is
0,1,1,0,0,0,0,0,0,0,1,...,1,1,1,0,0,1,0,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,0,0,3,1,1,0,1,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [ ] Obtener a traves de bag of words caracteristicas del resumen de historia de cada personaje.
- [ ] Aplicar MinMaxScaler sobre los atributos de interes.

**Respuesta:**

In [7]:
df_comics = df_comics.dropna(subset=['history_text'])

In [8]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
nltk.download('punkt')
bog = CountVectorizer(tokenizer= StemmerTokenizer(),  ngram_range=(1,2))
df = bog.fit_transform(df_comics["history_text"])

pd.DataFrame(df.toarray(), columns=bog.get_feature_names_out())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,!,! !,! '',! (,! ),"! ,",! --,! .,! ``,! alison,! almost,! amazon,! aquaman,! begin,! boil,! bud,! captur,! chang,! cheetah,! coalit,! crisi,! crossov,! decid,! demi-god,! diana,! dick,! featur,! first,! follow,! gener,! gone,! head,! initi,! insid,! jace,! jason,! kahn,! katherin,! know,! look,...,拳王 king,駄犬,"駄犬 ,",﻿juggernaut,﻿juggernaut recruit,�,� abomin,� batman,� bruce,� even,� hunt,� later,� much,� one,� prefer,� promis,� ray,� reactron,� similar,� skaar�,� would,�chain,�chain gang�,�doom,�doom old,�hellverine�,�hellverine� manag,�i,�i didn't.�,�kick�,�kick� .,�kick� driven,�polic,�polic man�,�white,�white room�,�ǣmortal,�ǣmortal instruments���,��wors,"��wors ,"
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1281,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1282,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1283,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
from sklearn.preprocessing import MinMaxScaler

atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
scaler = MinMaxScaler()
scaler.fit(df_comics[atributos_de_interes])
scaler.transform(df_comics[atributos_de_interes])

array([[0.85, 0.3 , 0.6 , 0.6 , 0.4 , 0.7 ],
       [0.8 , 1.  , 0.8 , 1.  , 1.  , 0.8 ],
       [0.8 , 0.5 , 0.55, 0.45, 1.  , 0.55],
       ...,
       [0.95, 0.5 , 1.  , 0.75, 1.  , 0.8 ],
       [0.75, 0.1 , 1.  , 0.3 , 1.  , 0.3 ],
       [0.45, 0.8 , 0.75, 0.95, 0.8 , 0.5 ]])

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>


Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Finalmente entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

**To-DO:**
- [ ] Realizar un pipeline con las caracteristicas solicitadas en 1.1 y aplicar un clasificador  `MultinomialNB()`.
- [ ] Entrenar el pipeline y comentar los resultados.

**Respuesta:**

In [54]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

preprocessing = ColumnTransformer(
                     transformers=[
                    ('df', CountVectorizer(
                    tokenizer=StemmerTokenizer(),
                    lowercase=True,  
                    max_features=10000,  
                    ngram_range=(1, 2)
                    ), 
                    'history_text'),
                    ('scaler', MinMaxScaler(), 
                    ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score'])
                  ]
)

In [47]:
from sklearn.feature_selection import (
    SelectKBest,
    SelectPercentile,
    f_classif,
    mutual_info_classif,
)

selection_pipeline = Pipeline(steps=[
    ('Preprocessing', preprocessing),
    ('Selection', SelectPercentile(f_classif, percentile=90)),
    ('NB', MultinomialNB()),
])

Entrenamiento

In [ ]:
X = df_comics[['history_text','intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']]
y = df_comics['alignment']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.30, random_state=90)

In [48]:
selection_pipeline.fit(X_train, y_train)
y_pred = selection_pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         Bad       0.54      0.69      0.61       130
        Good       0.78      0.70      0.74       230
     Neutral       0.21      0.12      0.15        26

    accuracy                           0.66       386
   macro avg       0.51      0.50      0.50       386
weighted avg       0.66      0.66      0.65       386



Se puede observar que las mejores métricas son para la clase "good", siguiendole las métricas de "Bad", sin embargo, para la clase "Neutral", son bastante malas, esto se puede deber quizás a algún desbalance de clases, esto podría estar influyendo en los resultados de clasificación, quizás hay un pequeño porcentaje de esta clase, que impida identificar bien las caracteristicas de esta clase. De froma general, se puede observar que el accuracy es bueno (pero podría mejorar)

¿Bat Cow buena , neutra o mala?

In [49]:
df_comics_no_label[df_comics_no_label["name"] == 'Batcow']

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,alter_egos,aliases,place_of_birth,first_appearance,creator,alignment,occupation,base,teams,relatives,gender,type_race,height,weight,eye_color,hair_color,skin_color,img,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,has_matter_manipulation,has_telepathy_resistance,has_mind_control,has_enhanced_hearing,has_dimensional_travel,has_element_control,...,has_fire_resistance,has_fire_control,has_dexterity,has_reality_warping,has_illusions,has_energy_beams,has_peak_human_condition,has_shapeshifting,has_heat_resistance,has_jump,has_self-sustenance,has_energy_absorption,has_cold_resistance,has_magic,has_telekinesis,has_toxin_and_disease_resistance,has_telepathy,has_regeneration,has_immortality,has_teleportation,has_force_fields,has_energy_manipulation,has_endurance,has_longevity,has_weapon-based_powers,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
122,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,20,10,20,"['Animal Attributes', 'Animal Oriented Powers']","['Milkman Man', 'Red Torpedo', 'Red Volcano']","[""Battlin' Bovine""]",NaN,NaN,DC Comics,NaN,NaN,Wayne Manor,[],NaN,NaN,Animal,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/13425.jpg?v=157425...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,20,10,20,"['Animal Attributes', 'Animal Oriented Powers']","['Milkman Man', 'Red Torpedo', 'Red Volcano']","[""Battlin' Bovine""]",NaN,NaN,DC Comics,NaN,NaN,Wayne Manor,[],NaN,NaN,Animal,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/13425.jpg?v=157425...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
y_test.iloc[122]  #Según el clasificador, este personaje es bueno

'Good'

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Dos clasificadores distintos en donde varie sus parámetros. Se le recomienda utilizar `LogisticRegression()` y `RandomForestClassifier()`.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. 
- Selección de las mejores columnas para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
        'clf__multi_class': ['liblinear']},
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [56]:
selection_pipeline = Pipeline(steps=[
                       ('preprocessing', preprocessing),
                       ('Selection', SelectPercentile(f_classif, percentile=20)),
                       ('clf', MultinomialNB())
                       ]
                ) 

In [57]:
selection_pipeline.get_params()


{'Selection': SelectPercentile(percentile=20),
 'Selection__percentile': 20,
 'Selection__score_func': <function sklearn.feature_selection._univariate_selection.f_classif>,
 'clf': MultinomialNB(),
 'clf__alpha': 1.0,
 'clf__class_prior': None,
 'clf__fit_prior': True,
 'memory': None,
 'preprocessing': ColumnTransformer(transformers=[('df',
                                  CountVectorizer(max_features=10000,
                                                  ngram_range=(1, 2),
                                                  tokenizer=<__main__.StemmerTokenizer object at 0x7f7c11e34190>),
                                  'history_text'),
                                 ('scaler', MinMaxScaler(),
                                  ['intelligence_score', 'strength_score',
                                   'speed_score', 'durability_score',
                                   'power_score', 'combat_score'])]),
 'preprocessing__df': CountVectorizer(max_features=10000, ngram_range=(1, 2

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.tree import DecisionTreeClassifier 

params = [
    {
        'preprocessing__df__max_features': [1000, 5000, 10000],
        'preprocessing__df__ngram_range': [(1,1), (1,2), (1,3)],
        'Selection__percentile': [20, 40, 60, 80],
        'Selection__score_func': [f_classif]
    },
    {
        'clf': [LogisticRegression()],
        'clf__multi_class': ['multinomial']
    },
    {
        'clf': [DecisionTreeClassifier()],
        'clf__criterion': ['gini', 'entropy']
        #'clf__n_estimators': [100],
        #'max_depth': np.arange(1,30),
    }
]

hgs = HalvingGridSearchCV(selection_pipeline, params ,random_state=0, verbose=10).fit(X_train, y_train)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 33
max_resources_: 899
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 39
n_resources: 33
Fitting 5 folds for each of 39 candidates, totalling 195 fits
[CV 1/5; 1/39] START Selection__percentile=20, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=1000, preprocessing__df__ngram_range=(1, 1)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


[CV 1/5; 1/39] END Selection__percentile=20, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=1000, preprocessing__df__ngram_range=(1, 1);, score=(train=0.846, test=0.667) total time=   0.4s
[CV 2/5; 1/39] START Selection__percentile=20, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=1000, preprocessing__df__ngram_range=(1, 1)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


[CV 2/5; 1/39] END Selection__percentile=20, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=1000, preprocessing__df__ngram_range=(1, 1);, score=(train=0.731, test=0.333) total time=   0.7s
[CV 3/5; 1/39] START Selection__percentile=20, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=1000, preprocessing__df__ngram_range=(1, 1)
[CV 3/5; 1/39] END Selection__percentile=20, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=1000, preprocessing__df__ngram_range=(1, 1);, score=(train=0.808, test=0.333) total time=   0.5s
[CV 4/5; 1/39] START Selection__percentile=20, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=1000, preprocessing__df__ngram_range=(1, 1)


...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [0.43333333 0.4        0.43333333 0.5        0.4               nan
 0.5               nan        nan 0.46666667 0.46666667 0.46666667
 0.5        0.5        0.5        0.5        0.5        0.5
 0.5        0.5        0.5        0.6        0.53333333 0.53333333
 0.6        0.5        0.5        0.63333333 0.6        0.6
 0.6        0.53333333 0.5        0.6        0.56666667 0.53333333
        nan        nan        nan 0.57894737 0.56842105 0.57894737
 0.56842105 0.56842105 0.56842105 0.58947368 0.56842105 0.41052632
 0.57894737 0.56842105 0.58947368 0.47368421]
  category=UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the train scores are non-finite: [0.86153846 0.83846154 0.84615385 0.81538462 0.80769231        nan
 0.81538462        nan        nan 0.91538462 0.93076923 0.9384

[CV 1/5; 1/5] END Selection__percentile=80, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=10000, preprocessing__df__ngram_range=(1, 1);, score=(train=0.962, test=0.678) total time=   5.0s
[CV 2/5; 1/5] START Selection__percentile=80, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=10000, preprocessing__df__ngram_range=(1, 1)
[CV 2/5; 1/5] END Selection__percentile=80, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=10000, preprocessing__df__ngram_range=(1, 1);, score=(train=0.932, test=0.525) total time=   5.2s
[CV 3/5; 1/5] START Selection__percentile=80, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=10000, preprocessing__df__ngram_range=(1, 1)
[CV 3/5; 1/5] END Selection__percentile=80, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=10000, preprocessing

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [0.43333333 0.4        0.43333333 0.5        0.4               nan
 0.5               nan        nan 0.46666667 0.46666667 0.46666667
 0.5        0.5        0.5        0.5        0.5        0.5
 0.5        0.5        0.5        0.6        0.53333333 0.53333333
 0.6        0.5        0.5        0.63333333 0.6        0.6
 0.6        0.53333333 0.5        0.6        0.56666667 0.53333333
        nan        nan        nan 0.57894737 0.56842105 0.57894737
 0.56842105 0.56842105 0.56842105 0.58947368 0.56842105 0.41052632
 0.57894737 0.56842105 0.58947368 0.47368421 0.62033898 0.62711864
 0.62033898 0.57627119 0.61694915]
  category=UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the train scores are non-finite: [0.86153846 0.83846154 0.84615385 0.81538462 0.80769231        nan
 0.815

[CV 1/5; 1/2] END Selection__percentile=20, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=10000, preprocessing__df__ngram_range=(1, 3);, score=(train=0.779, test=0.669) total time=  18.6s
[CV 2/5; 1/2] START Selection__percentile=20, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=10000, preprocessing__df__ngram_range=(1, 3)
[CV 2/5; 1/2] END Selection__percentile=20, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=10000, preprocessing__df__ngram_range=(1, 3);, score=(train=0.798, test=0.579) total time=  18.6s
[CV 3/5; 1/2] START Selection__percentile=20, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=10000, preprocessing__df__ngram_range=(1, 3)
[CV 3/5; 1/2] END Selection__percentile=20, Selection__score_func=<function f_classif at 0x7f7c260757a0>, preprocessing__df__max_features=10000, preprocessing

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [0.43333333 0.4        0.43333333 0.5        0.4               nan
 0.5               nan        nan 0.46666667 0.46666667 0.46666667
 0.5        0.5        0.5        0.5        0.5        0.5
 0.5        0.5        0.5        0.6        0.53333333 0.53333333
 0.6        0.5        0.5        0.63333333 0.6        0.6
 0.6        0.53333333 0.5        0.6        0.56666667 0.53333333
        nan        nan        nan 0.57894737 0.56842105 0.57894737
 0.56842105 0.56842105 0.56842105 0.58947368 0.56842105 0.41052632
 0.57894737 0.56842105 0.58947368 0.47368421 0.62033898 0.62711864
 0.62033898 0.57627119 0.61694915 0.61081699 0.62548086]
  category=UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the train scores are non-finite: [0.86153846 0.83846154 0.84615385 0.81538462 0.8076

In [67]:
hgs.cv_results_

{'iter': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3]),
 'mean_fit_time': array([ 0.38132734,  0.40175757,  0.452459  ,  0.37876606,  0.39842219,
         0.45464573,  0.37341199,  0.41308355,  0.46287122,  0.38233886,
         0.40197849,  0.44453206,  0.37307057,  0.4059144 ,  0.45219946,
         0.37643652,  0.40579267,  0.4580431 ,  0.38145332,  0.39966826,
         0.45373502,  0.37851024,  0.40236144,  0.44512758,  0.37813911,
         0.40532765,  0.45269012,  0.37121353,  0.40984969,  0.44824853,
         0.3751544 ,  0.41254439,  0.44709907,  0.37448773,  0.41081586,
         0.45870509,  0.44447222,  0.40692792,  0.41230664,  1.47630796,
         1.46941371,  1.47177162,  1.5980947 ,  1.80764041,  1.46440763,
         1.50110493,  1.6148859 ,  1.62897191,  1.81464338,  1.72052712,
         1.81244183,  1.61063046,  4.23953

In [84]:
Parametros = pd.DataFrame(hgs.cv_results_["params"])
Acc = pd.DataFrame(hgs.cv_results_["mean_test_score"], columns=["Accuracy"])
result = pd.concat([Parametros, Acc],axis=1)

In [85]:
result

,Selection__percentile,Selection__score_func,preprocessing__df__max_features,preprocessing__df__ngram_range,clf,clf__multi_class,clf__criterion,Accuracy
0,20.0,<function f_classif at 0x7f7c260757a0>,1000.0,"(1, 1)",NaN,NaN,NaN,0.433333
1,20.0,<function f_classif at 0x7f7c260757a0>,1000.0,"(1, 2)",NaN,NaN,NaN,0.400000
2,20.0,<function f_classif at 0x7f7c260757a0>,1000.0,"(1, 3)",NaN,NaN,NaN,0.433333
3,20.0,<function f_classif at 0x7f7c260757a0>,5000.0,"(1, 1)",NaN,NaN,NaN,0.500000
4,20.0,<function f_classif at 0x7f7c260757a0>,5000.0,"(1, 2)",NaN,NaN,NaN,0.400000
5,20.0,<function f_classif at 0x7f7c260757a0>,5000.0,"(1, 3)",NaN,NaN,NaN,NaN
6,20.0,<function f_classif at 0x7f7c260757a0>,10000.0,"(1, 1)",NaN,NaN,NaN,0.500000
7,20.0,<function f_classif at 0x7f7c260757a0>,10000.0,"(1, 2)",NaN,NaN,NaN,NaN
8,20.0,<function f_classif at 0x7f7c260757a0>,10000.0,"(1, 3)",NaN,NaN,NaN,NaN
9,40.0,<function f_classif at 0x7f7c260757a0>,1000.0,"(1, 1)",NaN,NaN,NaN,0.466667


Se puede observar que el mejor clasificador es el Logit del tipo multinomial, con respecto a sus parámetros, se observa que la configuración "preprocessing__df__ngram_range" = (1,1) y "preprocessing__df__max_features" = 1000 entregan el mejor acuracy.

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [65]:
df_comics_no_label = df_comics_no_label.dropna(subset=['history_text'])
df_comics_no_label = df_comics_no_label.drop_duplicates()
X_predict = df_comics_no_label[['history_text','intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']]

In [72]:
preprocessing= ColumnTransformer(
                    transformers=[
                    ('df', CountVectorizer(
                    tokenizer= StemmerTokenizer(),
                    lowercase=True,  
                    max_features=1000,  
                    ngram_range=(1, 1)
                    ), 
                    'history_text'),
                    ('scaler', MinMaxScaler(), 
                    ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score'])
                  ]
)

Pipeline_best = Pipeline(steps=[
                       ('preprocessing', preprocessing),
                       ('LR', LogisticRegression(multi_class = 'multinomial', max_iter = 1000))
                       ]
                )

Pipeline_best.fit(X_train, y_train)
y_pred = Pipeline_best.predict(X_predict)

In [74]:
df_comics_no_label['¿Qué son?'] = y_pred
personajes = ['Vergil' , 'Gorilla Girl' , 'Batcow']
df_comics_no_label[df_comics_no_label['name'].isin(personajes)]

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,power_score,combat_score,superpowers,alter_egos,aliases,place_of_birth,first_appearance,creator,alignment,occupation,base,teams,relatives,gender,type_race,height,weight,eye_color,hair_color,skin_color,img,has_electrokinesis,has_energy_constructs,has_mind_control_resistance,has_matter_manipulation,has_telepathy_resistance,has_mind_control,has_enhanced_hearing,has_dimensional_travel,has_element_control,...,has_fire_control,has_dexterity,has_reality_warping,has_illusions,has_energy_beams,has_peak_human_condition,has_shapeshifting,has_heat_resistance,has_jump,has_self-sustenance,has_energy_absorption,has_cold_resistance,has_magic,has_telekinesis,has_toxin_and_disease_resistance,has_telepathy,has_regeneration,has_immortality,has_teleportation,has_force_fields,has_energy_manipulation,has_endurance,has_longevity,has_weapon-based_powers,has_energy_blasts,has_enhanced_senses,has_invulnerability,has_stealth,has_marksmanship,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength,¿Qué son?
122,Batcow,None,None,3,Bat-Cow was originally a cow that was found by...,NaN,70,10,25,20,10,20,"['Animal Attributes', 'Animal Oriented Powers']","['Milkman Man', 'Red Torpedo', 'Red Volcano']","[""Battlin' Bovine""]",NaN,NaN,DC Comics,NaN,NaN,Wayne Manor,[],NaN,NaN,Animal,-,-,NaN,NaN,NaN,/pictures2/portraits/11/050/13425.jpg?v=157425...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Neutral
529,Gorilla Girl,Fahnbullah Eddy,Fahnbullah Eddy,7,A carnival performer with the ability to turn ...,Gorilla Girl can transform into a talking gori...,90,35,60,60,45,100,"['Agility', 'Jump', 'Super Strength', 'Transfo...",[],['Gorilla Woman'],NaN,NaN,Marvel Comics,NaN,NaN,NaN,[],NaN,NaN,NaN,-,-,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,Bad
1368,Vergil,Vergil Sparda,NaN,16,"Vergil, later also known as Nelo Angelo, is on...",NaN,90,75,95,90,100,100,"['Accelerated Healing', 'Agility', 'Duplicatio...",[],"['Gilver, Nelo Angelo, Son of Sparda, The Dark...",NaN,Devil May Cry,Capcom,NaN,NaN,NaN,[],"Sparda (father) ,Eva (mother) ,Dante (Brother)...",Male,NaN,6'3 • 191 cm,207 lb • 93 kg,Blue,White,NaN,/pictures2/portraits/10/050/11657.jpg?v=154990...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Good


Se observa que los personajes BatCow es predecido como "Neutral", "Gorilla Girl" como "mala"  y  "Vergil" como "bueno". Hay que notar que antes, el personaje "BatCow" se clasificó como un personaje bueno, pero al cambiar de clasificador y modificar algunos hiperparámetros, esta pasó de ser buena a ser neutral.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana y que **los días de atraso no se pueden utilizar para entregas de lab, solo para tareas**. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>